In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\1\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\1\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetMobile\1\NASNetMobile.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[13.0, 7.0, 40.0, 4.0, 84.0, 86.0, 4.0, 86.0, 80.0, 84.0, 96.0, 40.0, 7.0, 100.0, 93.0, 37.0, 55.0, 80.0, 84.0, 86.0, 95.0, 20.0, 37.0, 47.0, 47.0, 16.0, 84.0, 30.0, 86.0, 96.0, 16.0, 37.0, 93.0, 100.0, 45.0, 23.0, 47.0, 7.0, 86.0, 32.0, 7.0, 30.0, 23.0, 100.0, 95.0, 10.0, 30.0, 96.0, 100.0, 100.0, 95.0, 37.0, 13.0, 96.0, 13.0, 96.0, 47.0, 86.0, 13.0, 52.0, 37.0, 35.0, 52.0, 45.0, 45.0, 40.0, 37.0, 52.0, 10.0, 84.0, 55.0, 100.0, 35.0, 45.0, 10.0, 66.0, 13.0, 37.0, 37.0, 86.0, 45.0, 86.0, 80.0, 47.0, 95.0, 47.0, 40.0, 30.0, 47.0, 66.0, 84.0, 80.0, 100.0, 16.0, 47.0, 47.0, 66.0, 40.0, 4.0, 86.0, 84.0, 95.0, 84.0, 95.0, 16.0, 20.0, 30.0, 55.0, 4.0, 47.0, 37.0, 80.0, 30.0, 47.0, 37.0, 80.0, 7.0, 47.0, 100.0, 16.0, 80.0, 35.0, 10.0, 47.0, 16.0, 95.0, 40.0, 84.0, 32.0, 47.0, 84.0, 95.0, 13.0, 100.0, 66.0, 66.0, 47.0, 7.0, 84.0, 37.0, 84.0, 66.0, 37.0, 93.0, 10.0, 80.0, 37.0, 7.0, 23.0, 52.0, 30.0, 84.0, 80.0, 55.0, 37.0, 96.0, 4.0, 66.0, 84.0, 39.0, 80.0, 47.0, 95.0, 47.0, 23.0, 13.0, 16.0, 

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetMobile = tf.keras.applications.NASNetMobile(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetMobile.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetMobile(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (G  (None, 1056)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1057      
                                                                 
Total params: 4,270,773
Trainable params: 4,234,035
Non-trainable params: 36,738
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetMobile_loss_1.txt',history.history['loss'])
#要改
np.savetxt('NASNetMobile_valloss_1.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Fri Apr  7 15:20:13 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 70.2155

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\1\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\1\NASNetMobile.ckpt\assets


591/591 [==============================] - 178s 268ms/step - loss: 70.2155 - val_loss: 795.0545
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 2.5569

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\1\NASNetMobile.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\1\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\1\NASNetMobile.ckpt\assets


591/591 [==============================] - 209s 354ms/step - loss: 0.1861 - val_loss: 0.1154
Epoch 67/300
591/591 [==============================] - 128s 216ms/step - loss: 0.0460 - val_loss: 0.1262
Epoch 68/300
591/591 [==============================] - 126s 213ms/step - loss: 0.0512 - val_loss: 0.1850
Epoch 69/300
591/591 [==============================] - 126s 214ms/step - loss: 0.0456 - val_loss: 0.2116
Epoch 70/300
591/591 [==============================] - 126s 213ms/step - loss: 0.0485 - val_loss: 0.2282
Epoch 71/300
591/591 [==============================] - 126s 214ms/step - loss: 0.0605 - val_loss: 0.2198
Epoch 72/300
591/591 [==============================] - 126s 213ms/step - loss: 0.0685 - val_loss: 0.2398
Epoch 73/300
591/591 [==============================] - 126s 214ms/step - loss: 0.0446 - val_loss: 0.1638
Epoch 74/300
591/591 [==============================] - 127s 215ms/step - loss: 0.0421 - val_loss: 0.1621
Epoch 75/300
591/591 [==============================] - 127

591/591 [==============================] - 130s 220ms/step - loss: 0.0377 - val_loss: 0.1452
Epoch 144/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0386 - val_loss: 0.1559
Epoch 145/300
591/591 [==============================] - 130s 219ms/step - loss: 0.0325 - val_loss: 0.2014
Epoch 146/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0327 - val_loss: 0.2080
Epoch 147/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0342 - val_loss: 0.2397
Epoch 148/300
591/591 [==============================] - 131s 221ms/step - loss: 0.0802 - val_loss: 0.2951
Epoch 149/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0329 - val_loss: 0.2088
Epoch 150/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0352 - val_loss: 0.2787
Epoch 151/300
591/591 [==============================] - 130s 220ms/step - loss: 0.0447 - val_loss: 0.3119
Epoch 152/300
591/591 [============================

591/591 [==============================] - 135s 229ms/step - loss: 0.0261 - val_loss: 0.1815
Epoch 296/300
591/591 [==============================] - 135s 228ms/step - loss: 0.0238 - val_loss: 0.2037
Epoch 297/300
591/591 [==============================] - 135s 227ms/step - loss: 0.0244 - val_loss: 0.1584
Epoch 298/300
591/591 [==============================] - 135s 228ms/step - loss: 0.0381 - val_loss: 0.1931
Epoch 299/300
591/591 [==============================] - 136s 230ms/step - loss: 0.0240 - val_loss: 0.1564
Epoch 300/300
591/591 [==============================] - 135s 229ms/step - loss: 0.0236 - val_loss: 0.1671
end time: Sat Apr  8 02:26:57 2023
